# TOTVS Data Challenge

#### Exploratory Data Analysis

Our dataset is the transaction information of many restaurant meals. In this notebook we'll try and find interesting descriptions and patterns in the data.

In [37]:
import pymongo
import pprint

In [39]:
from connectionstring import connectionstring as cs
sample = pymongo.MongoClient(cs)['totvs']['sample']

In [40]:
# retrieving a random document and looking at each feature it has
doc = sample.aggregate(
   [{ '$sample': { 'size': 1 } }]
)

doc = list(doc)[0]

for k in doc.keys():
    if k != '_id':
        print('>', k, '\n')
        pprint.pprint(doc[k])
        print('\n--~--\n')

> complemento 

{'valorTotal': 28.47}

--~--

> dets 

[{'nItem': '1',
  'prod': {'indTot': '1',
           'qCom': 1.0,
           'uCom': 'UN',
           'vProd': 3.5,
           'vUnCom': 3.5,
           'xProd': 'AGUA'}},
 {'nItem': '2',
  'prod': {'indTot': '1',
           'qCom': 0.364,
           'uCom': 'KG',
           'vProd': 24.97,
           'vUnCom': 68.6,
           'xProd': 'BUFFET'}}]

--~--

> emit 

{'cnpj': '01.234.567/0001-89',
 'enderEmit': {'fone': '1-650-933-4902',
               'uf': 'CA',
               'xBairro': '',
               'xLgr': '650 Castro St. unit 210',
               'xMun': 'Mountain View',
               'xPais': 'United States'},
 'xFant': 'TOTVS Labs'}

--~--

> ide 

{'dhEmi': datetime.datetime(2016, 1, 12, 13, 1, 1), 'natOp': 'VENDA'}

--~--

> infAdic 

{'infCpl': 'Mesa 23'}

--~--

> total 

{'icmsTot': {'vDesc': 0.0,
             'vFrete': 0.0,
             'vOutro': 0.0,
             'vProd': 28.47,
             'vSeg': 0.0,
        

We can see we have data regarding how much the meal cost, discriminated by the food and drinks. What we unfortunely do not have is customer information, as it could be very important for predicting amount spent for example.

Let's check some descriptive statistics of our dataset.

In [51]:
# valorTotal
sample.aggregate([
    {
        '$group': {
            '_id': None,
            'meanValue': {'$avg': '$complemento.valorTotal'},
            'stdDeviation': {'$stdDevPop': '$complemento.valorTotal'},
            'maxValue': {'$max': '$complemento.valorTotal'},
            'minValue': {'$min': '$complemento.valorTotal'}
        }
    },{
        '$project': {
            '_id': 0
        }
    }
]).next()

{'maxValue': 608.91,
 'meanValue': 55.50417125382263,
 'minValue': 9.74,
 'stdDeviation': 39.20546729763231}